In [ ]:
import pandas as pd
import mlflow
import mlflow.sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
import numpy as np
import subprocess
import json

In [1]:
import pandas as pd
import numpy as np
import json

In [6]:
# Charger les données
df = pd.read_csv("data/creditcard.csv")
X = df.drop(["Time", "Class"], axis=1)

In [7]:
# Construire le JSON d'entrée (80 premières lignes)
input_data = X.iloc[:80].to_json(orient="split")

In [8]:
# Transformer en dict imbriqué
payload = {"dataframe_split": json.loads(input_data)}

In [10]:
print(type(payload))
payload

<class 'dict'>


{'dataframe_split': {'columns': ['V1',
   'V2',
   'V3',
   'V4',
   'V5',
   'V6',
   'V7',
   'V8',
   'V9',
   'V10',
   'V11',
   'V12',
   'V13',
   'V14',
   'V15',
   'V16',
   'V17',
   'V18',
   'V19',
   'V20',
   'V21',
   'V22',
   'V23',
   'V24',
   'V25',
   'V26',
   'V27',
   'V28',
   'Amount'],
  'index': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79],
  'data': [[-1.3598071337,
    -0.0727811733,
    2.536346738,
    1.3781552243,
    -0.3383207699,
    0.4623877778,
    0.2

In [11]:
# Envoyer la requête
import requests
response = requests.post(
    url="http://127.0.0.1:1235/invocations",
    headers={"Content-Type": "application/json"},
    json=payload
)

In [12]:
# Afficher la réponse
print(response.status_code)
print(response.json())

200
{'predictions': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


# Tester un modèle MLflow avec curl

In [18]:
# 1. Créer un fichier JSON d'entrée (ex : input.json)
input_data = X.iloc[:2].to_json(orient="split")  # exemple avec 2 lignes

with open("input.json", "w") as f:
    json.dump({"dataframe_split": json.loads(input_data)}, f)

In [ ]:
# 2. Appeler le modèle avec curl (a exécuter dans le terminal dans la racine du projet)
curl -X POST http://127.0.0.1:1235/invocations -H "Content-Type: application/json" -d @input.json


ou au lieu de taper la commande curl dans le terminal, on peut utiliser subprocess :

In [21]:
import subprocess

# Définir les paramètres
url = "http://127.0.0.1:1235/invocations"
headers = "Content-Type: application/json"
input_file = "input.json"

# Construire la commande curl
command = [
    "curl",
    "-X", "POST",
    url,
    "-H", headers,
    "-d", f"@{input_file}"
]

# Exécuter la commande
result = subprocess.run(command, capture_output=True, text=True)

# Afficher la réponse
print("Statut HTTP:", result.returncode)
print("Réponse:")
print(result.stdout)

# En cas d'erreur
if result.stderr:
    print("Erreur:")
    print(result.stderr)


Statut HTTP: 0
Réponse:
{"predictions": [0, 0]}
Erreur:
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1103  100    23  100  1080    687  32274 --:--:-- --:--:-- --:--:-- 33424



# Exemple avec dataframe_records aussi (plus lisible à l’œil nu) 

## 1. Génère un fichier input_records.json depuis Python

In [14]:
# Prendre 2 lignes et convertir au format records
input_data = X.iloc[:2].to_dict(orient="records")

# Envelopper dans le bon champ
payload = {"dataframe_records": input_data}

# Sauvegarder dans un fichier
with open("input_records.json", "w") as f:
    json.dump(payload, f, indent=2)

## 2. Appeler le modèle avec curl

In [ ]:
# (a exécuter dans le terminal dans la racine du projet)
curl -X POST http://127.0.0.1:1235/invocations -H "Content-Type: application/json" -d @input_records.json

Tu peux aussi éditer le fichier input_records.json à la main si tu veux tester des valeurs précises.

# Faire des prédictions sur une pandas.DataFrame avec un modèle MLflow

In [15]:
import pandas as pd
import requests
import json

# Charger les données
df = pd.read_csv("data/creditcard.csv")
X = df.drop(["Time", "Class"], axis=1).iloc[:2]  # Exemple sur 2 lignes

# Convertir la DataFrame au format dict (records)
data_dict = {"dataframe_records": X.to_dict(orient="records")}

# Envoyer la requête POST à l'API MLflow
response = requests.post(
    url="http://127.0.0.1:1235/invocations",
    headers={"Content-Type": "application/json"},
    data=json.dumps(data_dict)
)

# Afficher les prédictions
print("Statut:", response.status_code)
print("Prédictions:", response.json())

Statut: 200
Prédictions: {'predictions': [0, 0]}


# Wrapper Python réutilisable pour faire des prédictions avec une pandas.DataFrame via un modèle MLflow déployé en local (ou distant)

In [16]:
import pandas as pd
import requests
import json

def predict_local(df: pd.DataFrame, url: str = "http://127.0.0.1:1235/invocations"):
    """
    Envoie une DataFrame à un modèle MLflow déployé et retourne les prédictions.

    Args:
        df (pd.DataFrame): Les données d'entrée à prédire.
        url (str): L'URL du endpoint MLflow. Par défaut, localhost:1235.

    Returns:
        list: Les prédictions du modèle.
    """
    if not isinstance(df, pd.DataFrame):
        raise ValueError("L'entrée doit être une DataFrame pandas.")

    payload = {"dataframe_records": df.to_dict(orient="records")}

    headers = {"Content-Type": "application/json"}
    response = requests.post(url, headers=headers, data=json.dumps(payload))

    if response.status_code != 200:
        raise Exception(f"Erreur {response.status_code}: {response.text}")

    return response.json()

In [17]:
df = pd.read_csv("data/creditcard.csv").drop(["Time", "Class"], axis=1)
X_sample = df.iloc[:5]

preds = predict_local(X_sample)
print("Prédictions :", preds)

Prédictions : {'predictions': [0, 0, 0, 0, 0]}
